In [ ]:
from pyfedcamp import Reservations
import ipywidgets as widgets
import os
from IPython.display import display
import pandas as pd
from ipywidgets import VBox, HBox, Button, Output, Checkbox, Layout, SelectMultiple
from IPython.display import FileLink
import tempfile
import datetime

file_upload = widgets.FileUpload(
    accept='.xlsx',
    multiple=False,
    description='Upload file'
)

load_button = Button(description='Load File', button_style='primary')
output = Output()

def on_load_clicked(b):
    with output:
        output.clear_output()
        uploaded_files = file_upload.value
        if not uploaded_files:
            print("Please upload a file.")
            return
        try:
            upload_info = uploaded_files[0]
            temp_path = os.path.join(tempfile.gettempdir(), upload_info['name'])
            with open(temp_path, 'wb') as f:
                f.write(upload_info['content'])
            input_file = temp_path
            print(f"Loaded uploaded file: {input_file}")
            global res, upload_temp_path
            res = Reservations(input_file=input_file)
            upload_temp_path = temp_path  # Save for later use
            build_placard_form()
        except Exception as e:
            print(f"Error loading file: {e}")

load_button.on_click(on_load_clicked)

def build_placard_form():
    df = res.reservations[res.reservations['CheckInTag']]
    site_numbers = df['SiteNumber'].unique().tolist()
    site_numbers.sort()

    site_select = SelectMultiple(
        options=site_numbers,
        value=tuple(site_numbers),
        description='Campsites:',
        layout=Layout(width='50%')
    )

    select_all_checkbox = Checkbox(value=True, description='Select All Sites')

    def on_select_all_change(change):
        if change['new']:
            site_select.value = tuple(site_numbers)
        else:
            site_select.value = ()

    select_all_checkbox.observe(on_select_all_change, names='value')

    placard_output = Output()
    build_button = Button(description='Build Placards', button_style='success')
    download_link_box = Output()

    def on_build_clicked(b):
        with placard_output:
            placard_output.clear_output()
            download_link_box.clear_output()
            if not site_select.value:
                print("Please select at least one campsite.")
                return

            campsite_list = list(site_select.value)
            placard_records = df[
                (df['CheckInTag'] == True)
                &
                (df['SiteNumber'].isin(campsite_list))
            ][[
                'ReservationNumber',
                'SiteNumber',
                'ArrivalDate',
                'DepartureDate',
                'Occupants',
                'Primary Occupant Name'
            ]].to_dict(orient='records')

            # Get a date string for the filename (use earliest ArrivalDate in selection)
            date_str = pd.to_datetime(df['Arrival Date']).min().strftime('%Y-%m-%d')
            filename = f"{date_str}.pdf"
            output_path = upload_temp_path  # Use the same temp directory as upload
            pdf_path = os.path.join(os.path.dirname(output_path), filename)

            try:
                res.build_placards(
                    placard_records=placard_records,
                    filename=filename,
                    output_path=os.path.dirname(output_path)
                )
                print(f"Placards generated for sites:")
                for site in campsite_list:
                    print(f"- {site}")

                # Provide a download link
                with download_link_box:
                    display(FileLink(pdf_path, result_html_prefix="Download placards PDF: "))

            except Exception as e:
                print(f"Error: {e}")

    build_button.on_click(on_build_clicked)

    form = VBox([
        HBox([select_all_checkbox, site_select]),
        build_button,
        placard_output,
        download_link_box
    ])
    display(form)

display(VBox([file_upload, load_button, output]))